<center><img src="./imge/CLBLOGO.jpg" alt="创乐博" style="width: 300px;"/></center>

# 11.OpenCV 定位物体的实时位置

@－－－－湖南创乐博智能科技有限公司－－－－<br>
@  文件名：11.OpenCV定位物体的实时位置.ipynb <br>
@  版本：V2.0 <br>
@  author: zhulin<br>
@  说明：OpenCV定位物体的实时位置<br>

In [1]:
# 载入必要的库文件
from __future__ import print_function
from imutils.video import VideoStream
import imutils
import time
import cv2
import os
import RPi.GPIO as GPIO

In [2]:
# LED 初始化
redLed = 17
GPIO.setwarnings(False)
GPIO.setmode(GPIO.BCM)
GPIO.setup(redLed, GPIO.OUT)

In [3]:
# 打印物体的实时位置
def mapObjectPosition (x, y):
    print ("[INFO] Object Center coordenates at X0 = {0} and Y0 =  {1}".format(x, y))


In [4]:
dispW=500
dispH=350
flip=4
#调用CSI摄像头的属性参数
camSet=camSet='nvarguscamerasrc !  video/x-raw(memory:NVMM), width=3264, height=2464, format=NV12, framerate=21/1 ! nvvidconv flip-method='+str(flip)+' ! video/x-raw, width='+str(dispW)+', height='+str(dispH)+', format=BGRx ! videoconvert ! video/x-raw, format=BGR ! appsink'
# 初始化视频流并允许摄像机传感器预热
print("[INFO] waiting for camera to warmup...")
vs = cv2.VideoCapture(camSet) #VideoStream(camSet).start()
time.sleep(2.0)

[INFO] waiting for camera to warmup...


In [5]:
# 定义对象的上下边界
# 在HSV颜色空间中进行跟踪
colorLower = (9,135,231)
colorUpper = (31,255,255)

In [6]:
# 创建显示控件
from jetcam.utils import bgr8_to_jpeg
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
Frame = widgets.Image(format='jpeg', width=500, height=350)
display(Frame)

Image(value=b'', format='jpeg', height='350', width='500')

In [7]:
# 线程函数操作库
import threading # 线程
import ctypes
import inspect

# 线程结束代码
def _async_raise(tid, exctype):
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        raise SystemError("PyThreadState_SetAsyncExc failed")
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

In [8]:
# 关闭LED灯
GPIO.output(redLed, GPIO.LOW)
ledOn = False

In [9]:
def Video_display():
    global ledOn
    # 循环的帧从视频流
    while True:
        # 从视频流中抓取下一帧，调整大小
        # 帧，并将其转换为HSV颜色空间
        ret, frame = vs.read()
        frame = imutils.resize(frame, width=500)
        frame = imutils.rotate(frame, angle=0)
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # 为对象颜色构造一个遮罩，然后执行
        # 一系列的膨胀和侵蚀，以消除任何小的
        # blobs left in the mask
        mask = cv2.inRange(hsv, colorLower, colorUpper)
        mask = cv2.erode(mask, None, iterations=2)
        mask = cv2.dilate(mask, None, iterations=2)

        # 找到遮罩中的轮廓并初始化
        # (x, y) center of the object
        cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2]
        #cnts = cnts[0] if imutils.is_cv2() else cnts[1]
        center = None

        # 只有在找到至少一条轮廓线时才进行
        if len(cnts) > 0:
            # 在蒙版中找到最大的轮廓，然后使用
            # 它可以计算出最小的围圆
            # 重心
            c = max(cnts, key=cv2.contourArea)
            ((x, y), radius) = cv2.minEnclosingCircle(c)
            M = cv2.moments(c)
            center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

            # 只有当半径满足最小尺寸时才进行
            if radius > 10:
                # 在框架上画圆和质心，
                # 然后更新跟踪点的列表
                cv2.circle(frame, (int(x), int(y)), int(radius),(0, 255, 255), 2)
                cv2.circle(frame, center, 5, (0, 0, 255), -1)

                # 定位舵机在圆心
                mapObjectPosition(int(x), int(y))

                # 如果led还没有打开，打开led
                if not ledOn:
                    GPIO.output(redLed, GPIO.HIGH)
                    ledOn = True
        # 如果没有检测到球，关闭LED灯
        elif ledOn:
            GPIO.output(redLed, GPIO.LOW)
            ledOn = False

        # 向我们的屏幕显示框架
        Frame.value = bgr8_to_jpeg(frame)    
    # 做点清理工作
    print("\n [INFO] Exiting Program and cleanup stuff \n")
    GPIO.cleanup()
    vs.stop()

In [10]:
# 开始线程
t = threading.Thread(target=Video_display)
t.setDaemon(True)
t.start()

[INFO] Object Center coordenates at X0 = 202 and Y0 =  169
[INFO] Object Center coordenates at X0 = 208 and Y0 =  169
[INFO] Object Center coordenates at X0 = 224 and Y0 =  165
[INFO] Object Center coordenates at X0 = 232 and Y0 =  155
[INFO] Object Center coordenates at X0 = 233 and Y0 =  151
[INFO] Object Center coordenates at X0 = 229 and Y0 =  150
[INFO] Object Center coordenates at X0 = 222 and Y0 =  150
[INFO] Object Center coordenates at X0 = 169 and Y0 =  204
[INFO] Object Center coordenates at X0 = 176 and Y0 =  203
[INFO] Object Center coordenates at X0 = 175 and Y0 =  200
[INFO] Object Center coordenates at X0 = 173 and Y0 =  197
[INFO] Object Center coordenates at X0 = 172 and Y0 =  191
[INFO] Object Center coordenates at X0 = 172 and Y0 =  186
[INFO] Object Center coordenates at X0 = 172 and Y0 =  182
[INFO] Object Center coordenates at X0 = 172 and Y0 =  181
[INFO] Object Center coordenates at X0 = 176 and Y0 =  186
[INFO] Object Center coordenates at X0 = 231 and Y0 =  1

In [11]:
# 结束线程
stop_thread(t)